In [1]:
from src.clp_zne.qpu_info.layout_cycles import heron_layout_cycles_12q
from src.clp_zne.hamiltonians import sherrington_kirkpatrick_model
from src.clp_zne.qpu_info.backends import FakeTorino
from src.clp_zne.mitigate import zne_mitigate, clp_zne_mitigate_general_topology_circuit
from src.clp_zne.backends import NClusterBackend
from src.clp_zne.utils import compute_evals_ideal, get_grid_entanglement
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.library import TwoLocal
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
rows, cols = 3, 3
num_qubits = rows * cols

entanglement_map = get_grid_entanglement(rows, cols)

ansatz = TwoLocal(num_qubits, ['rx', 'rz'], 'cz', entanglement=entanglement_map, reps=3)

In [7]:
source_backend = FakeTorino()
backend = NClusterBackend.from_backend(cluster_size=num_qubits, source_backend=source_backend,
                                         list_of_index_groups=[heron_layout_cycles_12q[i] for i in [5, 9, 13]], seed=42)

observables = [sherrington_kirkpatrick_model(num_qubits, h=1, seed=i) for i in range(100)]

circuits = []
for i in range(20):
    circuit = ansatz.copy()
    rng = np.random.default_rng(i)
    parameters = rng.uniform(-np.pi, np.pi, circuit.num_parameters)
    circuit.assign_parameters(parameters, inplace=True)
    circuits.append(circuit)

In [8]:
noise_model = NoiseModel.from_backend(backend, readout_error=False, gate_error=True, thermal_relaxation=True)

layouts = [heron_layout_cycles_12q[i] for i in [5, 9, 13, 16, 7]]

all_evals_ideal = []
all_evals_mitigated = []
all_evals_noisy = []
all_error_sums = []

for circ in tqdm(circuits):
    evals_ideal = compute_evals_ideal(circ, observables)
    evals_mitigated, evals_noisy, error_sums = clp_zne_mitigate_general_topology_circuit(
                                                circ, observables,
                                                np.arange(backend.num_qubits).reshape((backend.num_clusters, -1)).tolist(), 
                                                backend, noise_model
                                                )
    all_evals_ideal.append(evals_ideal)
    all_evals_mitigated.append(evals_mitigated)
    all_evals_noisy.append(evals_noisy)
    all_error_sums.append(error_sums)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 41%|████████████████████████████████▊                                               | 41/100 [00:00<00:00, 405.94it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 409.83it/s]

  4%|███                                                                                | 1/27 [00:00<00:11,  2.27it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 43%|██████████████████████████████████▍                                             | 43/100 [00:00<00:00, 421.57it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 401.21it/s]

  7%|██████▏                   

 59%|████████████████████████████████████████████████▌                                 | 16/27 [00:07<00:04,  2.21it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 42%|█████████████████████████████████▌                                              | 42/100 [00:00<00:00, 415.84it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 408.10it/s]

 63%|███████████████████████████████████████████████████▋                              | 17/27 [00:07<00:04,  2.24it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 40%|████████████████████████████████                                                | 40/100 [00:00<00:00, 399.88it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 396.82it/s]

 67%|███████████████████████████

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations


...


  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 43%|██████████████████████████████████▍                                             | 43/100 [00:00<00:00, 421.57it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 412.46it/s]

  4%|███                                                                                | 1/27 [00:00<00:11,  2.30it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 41%|████████████████████████████████▊                                               | 41/100 [00:00<00:00, 405.83it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 408.11it/s]

  7%|██████▏                   

 59%|████████████████████████████████████████████████▌                                 | 16/27 [00:07<00:05,  2.20it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 41%|████████████████████████████████▊                                               | 41/100 [00:00<00:00, 409.87it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 401.56it/s]

 63%|███████████████████████████████████████████████████▋                              | 17/27 [00:07<00:04,  2.22it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 42%|█████████████████████████████████▌                                              | 42/100 [00:00<00:00, 415.72it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 396.78it/s]

 67%|███████████████████████████

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 41%|████████████████████████████████▊                                               | 41/100 [00:00<00:00, 401.96it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 396.78it/s]

  4%|███                                                                                | 1/27 [00:00<00:11,  2.26it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 42%|█████████████████████████████████▌                                              | 42/100 [00:00<00:00, 407.77it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 414.81it/s]

  7%|██████▏                   

 59%|████████████████████████████████████████████████▌                                 | 16/27 [00:07<00:05,  2.17it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 41%|████████████████████████████████▊                                               | 41/100 [00:00<00:00, 410.00it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 395.26it/s]

 63%|███████████████████████████████████████████████████▋                              | 17/27 [00:07<00:04,  2.20it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 42%|█████████████████████████████████▌                                              | 42/100 [00:00<00:00, 407.77it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 411.52it/s]

 67%|███████████████████████████

In [9]:
import os
import numpy as np

# Create folder for experimental data
folder_name = "data\SK_histogram_square_topology_full_noise"
os.makedirs(folder_name, exist_ok=True)

# Convert to numpy arrays (if they aren't already)
all_evals_ideal_np = np.array(all_evals_ideal)
all_evals_mitigated_np = np.array(all_evals_mitigated)
all_evals_noisy_np = np.array(all_evals_noisy)
all_error_sums_np = np.array(all_error_sums)

# Save the arrays in the folder
np.save(os.path.join(folder_name, 'all_evals_ideal.npy'), all_evals_ideal_np)
np.save(os.path.join(folder_name, 'all_evals_mitigated.npy'), all_evals_mitigated_np)
np.save(os.path.join(folder_name, 'all_evals_noisy.npy'), all_evals_noisy_np)
np.save(os.path.join(folder_name, 'all_error_sums.npy'), all_error_sums_np)

print(f"Arrays saved successfully in the '{folder_name}' folder.")

Arrays saved successfully in the 'data\SK_histogram_square_topology_full_noise' folder.


<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Z.Sayapin\AppData\Local\Temp\ipykernel_18852\4158979430.py:5: SyntaxWarning: invalid escape sequence '\S'
  folder_name = "data\SK_histogram_square_topology_full_noise"
